In [2]:
from nba_api.stats.static import teams
from team_stats_helpers import find_team_id

team_id = find_team_id()
team_id

[[1610612737, 'Atlanta Hawks'],
 [1610612738, 'Boston Celtics'],
 [1610612739, 'Cleveland Cavaliers'],
 [1610612740, 'New Orleans Pelicans'],
 [1610612741, 'Chicago Bulls'],
 [1610612742, 'Dallas Mavericks'],
 [1610612743, 'Denver Nuggets'],
 [1610612744, 'Golden State Warriors'],
 [1610612745, 'Houston Rockets'],
 [1610612746, 'Los Angeles Clippers'],
 [1610612747, 'Los Angeles Lakers'],
 [1610612748, 'Miami Heat'],
 [1610612749, 'Milwaukee Bucks'],
 [1610612750, 'Minnesota Timberwolves'],
 [1610612751, 'Brooklyn Nets'],
 [1610612752, 'New York Knicks'],
 [1610612753, 'Orlando Magic'],
 [1610612754, 'Indiana Pacers'],
 [1610612755, 'Philadelphia 76ers'],
 [1610612756, 'Phoenix Suns'],
 [1610612757, 'Portland Trail Blazers'],
 [1610612758, 'Sacramento Kings'],
 [1610612759, 'San Antonio Spurs'],
 [1610612760, 'Oklahoma City Thunder'],
 [1610612761, 'Toronto Raptors'],
 [1610612762, 'Utah Jazz'],
 [1610612763, 'Memphis Grizzlies'],
 [1610612764, 'Washington Wizards'],
 [1610612765, 'Det

# Get All Recorded Games in NBA History

In [76]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

all_games_list = []

for i in range(len(team_id)):
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id[i][0])
    games = gamefinder.get_data_frames()[0]

    all_games_list.append(games)

all_games_df = pd.concat(all_games_list)

# Joins All Team Games into One Matchup Stat

In [56]:
import pandas as pd

def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result
    
# Combine the game rows into one. By default, the home team will be TEAM_A.
combined_all_games_df = combine_team_games(all_games_df)

In [5]:
import pandas as pd
combined_all_games_df = pd.read_csv("/home/rohanbendapudi/github_repos/nba_predictor_development/models/data/all_games.csv")

In [57]:
def find_matchups(all_games_df, team_a, team_b):
    """
    Find all games where to specific teams go against each other
    """
    matchup_1 = all_games_df[(all_games_df["TEAM_ID_A"] == team_a) & (all_games_df["TEAM_ID_B"] == team_b)]
    matchup_2 = all_games_df[(all_games_df["TEAM_ID_A"] == team_b) & (all_games_df["TEAM_ID_B"] == team_a)]

    all_matchups = pd.concat([matchup_1, matchup_2])

    return all_matchups

all_matchups = find_matchups(combined_all_games_df, int('1610612737'),int('1610612766'))

,SEASON_ID,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE,MATCHUP_A,WL_A,MIN_A,PTS_A,FGM_A,FGA_A,FG_PCT_A,FG3M_A,FG3A_A,FG3_PCT_A,FTM_A,FTA_A,FT_PCT_A,OREB_A,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLUS_MINUS_A,TEAM_ID_B,TEAM_ABBREVIATION_B,TEAM_NAME_B,MATCHUP_B,WL_B,MIN_B,PTS_B,FGM_B,FGA_B,FG_PCT_B,FG3M_B,FG3A_B,FG3_PCT_B,FTM_B,FTA_B,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B
7,22023,1610612737,ATL,Atlanta Hawks,0022301159,2024-04-10,ATL vs. CHA,L,240,114,43,82,0.524,18,40.0,0.450,10,16,0.625,7.0,31.0,38.0,35,7.0,2,16,20,-1.0,1610612766,CHA,Charlotte Hornets,CHA @ ATL,W,241,115,40,79,0.506,14,36.0,0.389,21,22,0.955,5.0,28.0,33.0,25,11.0,2,13,18,1.0
27,22023,1610612737,ATL,Atlanta Hawks,0022301021,2024-03-23,ATL vs. CHA,W,240,132,50,90,0.556,20,42.0,0.476,12,14,0.857,11.0,32.0,43.0,40,9.0,5,8,14,41.0,1610612766,CHA,Charlotte Hornets,CHA @ ATL,L,240,91,35,84,0.417,11,39.0,0.282,10,14,0.714,11.0,28.0,39.0,25,3.0,4,13,9,-41.0
271,22022,1610612737,ATL,Atlanta Hawks,0022200694,2023-01-21,ATL vs. CHA,L,240,118,45,83,0.542,11,25.0,0.440,17,20,0.850,6.0,34.0,40.0,24,5.0,4,16,20,-4.0,1610612766,CHA,Charlotte Hornets,CHA @ ATL,W,239,122,45,89,0.506,13,34.0,0.382,19,22,0.864,6.0,30.0,36.0,28,11.0,2,12,18,4.0
359,22022,1610612737,ATL,Atlanta Hawks,0022200038,2022-10-23,ATL vs. CHA,L,240,109,39,95,0.411,8,35.0,0.229,23,28,0.821,16.0,29.0,45.0,23,8.0,9,12,21,-17.0,1610612766,CHA,Charlotte Hornets,CHA @ ATL,W,240,126,47,90,0.522,11,26.0,0.423,21,30,0.700,14.0,38.0,52.0,28,7.0,6,13,24,17.0
399,52021,1610612737,ATL,Atlanta Hawks,0052100111,2022-04-13,ATL vs. CHA,W,240,132,49,94,0.521,16,32.0,0.500,18,25,0.720,12.0,42.0,54.0,31,5.0,6,10,25,29.0,1610612766,CHA,Charlotte Hornets,CHA @ ATL,L,239,103,34,90,0.378,13,41.0,0.317,22,29,0.759,11.0,30.0,41.0,22,4.0,4,14,24,-29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210988,21990,1610612766,CHH,Charlotte Hornets,0029000982,1991-04-04,CHH vs. ATL,W,240,98,38,78,0.487,1,3.0,0.333,21,30,0.700,9.0,27.0,36.0,27,13.0,6,14,20,NaN,1610612737,ATL,Atlanta Hawks,ATL @ CHH,L,240,91,35,89,0.393,1,13.0,0.077,20,28,0.714,21.0,30.0,51.0,14,3.0,6,20,26,NaN
211112,21990,1610612766,CHH,Charlotte Hornets,0029000124,1990-11-20,CHH vs. ATL,W,240,128,48,90,0.533,3,5.0,0.600,29,39,0.744,19.0,32.0,51.0,37,15.0,7,20,29,NaN,1610612737,ATL,Atlanta Hawks,ATL @ CHH,L,240,121,41,85,0.482,4,12.0,0.333,35,47,0.745,15.0,23.0,38.0,33,12.0,5,19,29,NaN
211224,21989,1610612766,CHH,Charlotte Hornets,0028900518,1990-01-23,CHH vs. ATL,L,240,101,39,94,0.415,2,7.0,0.286,21,29,0.724,15.0,31.0,46.0,26,6.0,6,10,28,NaN,1610612737,ATL,Atlanta Hawks,ATL @ CHH,W,240,106,33,83,0.398,0,4.0,0.000,40,47,0.851,15.0,38.0,53.0,18,7.0,4,12,23,NaN
211314,21988,1610612766,CHH,Charlotte Hornets,0028800937,1989-04-10,CHH vs. ATL,L,240,105,42,92,0.457,1,7.0,0.143,20,23,0.870,14.0,29.0,43.0,34,8.0,2,11,35,NaN,1610612737,ATL,Atlanta Hawks,ATL @ CHH,W,240,112,34,85,0.400,1,3.0,0.333,43,54,0.796,23.0,30.0,53.0,26,5.0,7,12,18,NaN


# Get Available Box-Scores Based on Game_ID

In [14]:
from nba_api.stats.endpoints import boxscoretraditionalv2

box_score = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id='0022301159')
# box_score = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id='0022301159')
box_score.get_data_frames()[0]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0022301159,1610612766,CHA,Charlotte,1641706,Brandon Miller,Brandon,F,,33.000000:24,...,0.0,3.0,3.0,2.0,0.0,0.0,1.0,2.0,27.0,1.0
1,0022301159,1610612766,CHA,Charlotte,1628970,Miles Bridges,Miles,F,,29.000000:51,...,1.0,2.0,3.0,3.0,0.0,0.0,2.0,0.0,18.0,-9.0
2,0022301159,1610612766,CHA,Charlotte,1629684,Grant Williams,Grant,C,,26.000000:33,...,1.0,5.0,6.0,5.0,1.0,0.0,3.0,5.0,16.0,-1.0
3,0022301159,1610612766,CHA,Charlotte,1630544,Tre Mann,Tre,G,,36.000000:45,...,1.0,2.0,3.0,7.0,7.0,0.0,1.0,0.0,16.0,2.0
4,0022301159,1610612766,CHA,Charlotte,203995,Vasilije Micic,Vasilije,G,,21.000000:43,...,0.0,2.0,2.0,5.0,1.0,0.0,2.0,0.0,2.0,-14.0
5,0022301159,1610612766,CHA,Charlotte,1630197,Aleksej Pokusevski,Aleksej,,,25.000000:31,...,0.0,6.0,6.0,1.0,1.0,1.0,1.0,2.0,10.0,5.0
6,0022301159,1610612766,CHA,Charlotte,1630208,Nick Richards,Nick,,,18.000000:44,...,0.0,2.0,2.0,1.0,1.0,0.0,1.0,6.0,7.0,1.0
7,0022301159,1610612766,CHA,Charlotte,202722,Davis Bertans,Davis,,,18.000000:09,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,6.0,10.0
8,0022301159,1610612766,CHA,Charlotte,1630550,JT Thor,JT,,,18.000000:09,...,2.0,2.0,4.0,1.0,0.0,0.0,1.0,2.0,6.0,10.0
9,0022301159,1610612766,CHA,Charlotte,1631121,Bryce McGowens,Bryce,,,8.000000:28,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,-1.0


In [35]:
from stat_score import StatScore
from collections import deque

hawks_team_id = team_id[0][0]

hawks_dict = {"team_id":{}}

# get 5 previous games
previous_games_queue = deque()
previous_games_df = combined_all_games_df[(combined_all_games_df["TEAM_ID_A"]==hawks_team_id) | (combined_all_games_df["TEAM_ID_B"]==hawks_team_id)].head(10)

for i, row in previous_games_df.iterrows():
    game_ID = '00'+str(row["GAME_ID"])
    
    print(game_ID)
    previous_games_queue.append(StatScore(game_ID=game_ID,all_games=combined_all_games_df))

previous_games_queue

0022301159
0022301147
0022301104
0022301076
0022301060
0022301051
0022301034
0022301021
0022300925
0022300895


deque([<stat_score.StatScore at 0x7a0cb6c47e20>,
       <stat_score.StatScore at 0x7a0cb6c67c10>])

In [37]:
previous_games = hawks_dict["team_id"]
previous_games["PREVIOUS_GAMES"] = previous_games_queue
previous_games["PREVIOUS_GAMES"] 

deque([<stat_score.StatScore at 0x7a0cb6c47e20>,
       <stat_score.StatScore at 0x7a0cb6c67c10>])